# Coletando as páginas de cada vídeo - Etapa 2
Nessa fase, iremos coletar as páginas individuais de cada vídeo presente na busca inicial (notebook coleta_de_dados.ipnynb).
A partir dos dados de cada vídeo será possível coletar as features desejadas para criação do modelo.

In [5]:
#importando pacotes
import pandas as pd
import numpy as np
import re
import time

import requests as rq
import bs4
from tqdm import tqdm
import glob
import json

### 1 - Carregando a lista de páginas

In [9]:
df_lista = pd.read_json(".\datasets\parsed_videos.json", lines=True)
df_lista.head()

,link,title,query
0,/watch?v=3XLlu8GC2gg,URGENTE! A TAXA SELIC CAIU PARA 3%! | QUAL O I...,investimentos
1,/watch?v=-sGOvusS5NI,3 INVESTIMENTOS PRA INICIANTES NA BOLSA DE VAL...,investimentos
2,/watch?v=uuvGLCQUvFY,Taxa Selic caiu mais uma vez! Onde investir ne...,investimentos
3,/watch?v=-HJn3vgOx_M,GANHE DINHEIRO EM 2020! TOP 6 investimentos do...,investimentos
4,/watch?v=dJyJ77GkhBE,O Guia BÁSICO para começar a INVESTIR com POUC...,investimentos


In [10]:
#quantidade de links:
len(df_lista['link'])

1600

In [11]:
#verificando presença de links duplicados.
len(df_lista['link'])-len(df_lista['link'].unique())

82

Temos links duplicados. Logo, vamos criar um novo dataframe com links únicos.

In [12]:
links = df_lista['link'].unique()
len(links)

1518

In [19]:
links[0:3]

array(['/watch?v=3XLlu8GC2gg', '/watch?v=-sGOvusS5NI',
       '/watch?v=uuvGLCQUvFY'], dtype=object)

Agora podemos continuar para a coleta das páginas de cada link no youtube.

In [13]:
# criando raiz da url de cada video
url = "https://www.youtube.com{link}"

In [24]:
# coletando páginas de cada video 
for link in tqdm(links):
    urll = url.format(link=link)
    response = rq.get(urll)
    link_name = re.search("v=(.*)", link).group(1) #será o nome do arquivo html salvo
    
    with open("./paginas_videos/video_{}.html".format(link_name), 'w+', encoding='utf8') as output:
        output.write(response.text)
    time.sleep(2)

100%|████████████████████████████████████████████████████████████████████████████| 1518/1518 [1:19:09<00:00,  3.13s/it]


## Processamento dos dados das páginas
Nessa fase, iremos extrair features de cada página de vídeo obtida na fase anterior e salva em html na pasta .\paginas_videos. Nesse momento, conforme abordagem seguida pelo instrutor do curso, serão baixados diversos atributos das páginas, que serão depois triados para elaborar um modelo inicial.

In [6]:
with open ("./datasets/parsed_videos_info.json", 'w+') as output: #esse json irá conter todas as features extraídas
    for file in tqdm(sorted(glob.glob("./paginas_videos/video*"))): #matching todos os arquivos que iniciam com 'video'
        with open(file, 'r+', encoding='utf8') as inp:
            page = inp.read()
            parsed = bs4.BeautifulSoup(page,'html.parser')
            
            class_watch = parsed.find_all(attrs={"class":re.compile(r"watch")}) #todas as tags class que contém a string 'watch'
            id_watch = parsed.find_all(attrs={"id":re.compile(r"watch")}) #todas as tags id que possuem watch
            channel = parsed.find_all("a", attrs={"href":re.compile(r"channel")})
            meta = parsed.find_all("meta")
            
            #criando dicionário que irá ser salvo como json
            data = dict ()
            
            for e in class_watch: #elementos no objeto class_watch
                colname = "_".join(e['class'])
                if "clearfix" in colname: # se o atributo clearfix existir, ignoramos (aparentemente, cria várias colunas para o mesmo viídeo)
                    continue
                data[colname] = e.text.strip()
            
            for e in id_watch:
                colname = e['id']
                data[colname] = e.text.strip()
                
            for e in meta:
                colname = e.get('property')
                if colname is not None:
                    data[colname] = e['content']
            
            for link_num, e in enumerate(channel):
                data["channel_link_{}".format(link_num)] = e['href']
                
            output.write("{}\n".format(json.dumps(data)))
            
                
                
            

100%|██████████████████████████████████████████████████████████████████████████████| 1518/1518 [06:17<00:00,  4.02it/s]


## Verificação e seleção inicial de features

Nessa fase, iremos converter o arquivo .json com os atributos extraídos anteriormente em um dataframe do pandas para fazer uma verificação inicial.

In [9]:
df_raw = pd.read_json('./datasets/parsed_videos_info.json', lines=True)
df_raw.shape

(1518, 127)

In [14]:
# número de colunas = 127. Para inspeção manual, é necessário visualizar todas as colunas.
pd.set_option('display.max_columns', 127)
df_raw.head(2)

,content-alignment_watch-small,watch-playlist_player-height,watch-queue-header,watch-queue-info,watch-queue-info-icon,watch-queue-title,watch-queue-control-bar_control-bar-button,watch-queue-mole-info,watch-queue-control-bar-icon,watch-queue-icon_yt-sprite,watch-queue-title-container,watch-queue-count,watch-queue-menu_yt-uix-button-menu_yt-uix-button-menu-dark-overflow-action-menu_hid,watch-queue-menu-choice_overflow-menu-choice_yt-uix-button-menu-item,watch-queue-controls,yt-uix-button_yt-uix-button-size-default_yt-uix-button-empty_yt-uix-button-has-icon_control-bar-button_prev-watch-queue-button_yt-uix-button-opacity_yt-uix-tooltip_yt-uix-tooltip,yt-uix-button-icon_yt-uix-button-icon-watch-queue-prev_yt-sprite,yt-uix-button_yt-uix-button-size-default_yt-uix-button-empty_yt-uix-button-has-icon_control-bar-button_play-watch-queue-button_yt-uix-button-opacity_yt-uix-tooltip_yt-uix-tooltip,yt-uix-button-icon_yt-uix-button-icon-watch-queue-play_yt-sprite,yt-uix-button_yt-uix-button-size-default_yt-uix-button-empty_yt-uix-button-has-icon_control-bar-button_pause-watch-queue-button_yt-uix-button-opacity_yt-uix-tooltip_hid_yt-uix-tooltip,yt-uix-button-icon_yt-uix-button-icon-watch-queue-pause_yt-sprite,yt-uix-button_yt-uix-button-size-default_yt-uix-button-empty_yt-uix-button-has-icon_control-bar-button_next-watch-queue-button_yt-uix-button-opacity_yt-uix-tooltip_yt-uix-tooltip,yt-uix-button-icon_yt-uix-button-icon-watch-queue-next_yt-sprite,watch-queue-items-container_yt-scrollbar-dark_yt-scrollbar,watch-queue-items-list,content-alignment_watch-player-playlist,watch-main-col,watch-title-container,watch-title,watch-secondary-actions_yt-uix-button-group,watch-view-count,watch-action-panels_yt-uix-button-panel_hid_yt-card_yt-card-has-padding,watch-time-text,watch-extras-section,watch-meta-item_yt-uix-expander-body,content_watch-info-tag-list,watch-sidebar,watch-playlist_player-height_hid,watch-sidebar-gutter_yt-card_yt-card-has-padding_yt-uix-expander_yt-uix-expander-collapsed,watch-sidebar-section,watch-sidebar-head,watch-sidebar-body,watch-sidebar-separation-line,watch-queue-mole,watch-queue,watch-queue-title-msg,watch-queue-count-msg,watch-queue-loading-template,watch7-container,watch7-main-container,watch7-main,watch7-preview,watch7-content,watch-header,watch7-headline,watch-headline-title,watch7-user-header,watch7-subscription-container,watch8-action-buttons,watch8-secondary-actions,watch8-sentiment-actions,watch7-views-info,watch-action-panels,watch-actions-share-loading,watch-actions-share-panel,watch-actions-transcript-loading,watch-actions-transcript,watch-transcript-container,watch-transcript-not-found,watch-actions-rental-required,watch-description,watch-description-content,watch-description-clip,watch-uploader-info,watch-description-text,watch-description-extras,watch-discussion,watch7-sidebar,watch7-sidebar-contents,watch7-sidebar-offer,watch7-sidebar-ads,watch7-sidebar-modules,watch-related,shared-addto-watch-later-login,og:site_name,og:url,og:title,og:image,og:image:width,og:image:height,og:description,al:ios:app_store_id,al:ios:app_name,al:ios:url,al:android:url,al:android:app_name,al:android:package,al:web:url,og:type,og:video:url,og:video:secure_url,og:video:type,og:video:width,og:video:height,og:video:tag,fb:app_id,channel_link_0,channel_link_1,channel_link_2,watch-channel-brand-div,watch-channel-brand-div-text,content-alignment_watch-small_off-screen-trigger,watch7-player-age-gate-content,yt-pl-watch-queue-overlay,watch-meta-item,watch-meta-item_has-image,channel_link_3,channel_link_4,channel_link_5,channel_link_6,channel_link_7,watch-sidebar-live-chat,watch-skeleton,watch-page-skeleton,watch-sidebar-discussion,watch-checkout-offers,og:restrictions:age
0,Este vídeo não está disponível.\n\n \n\n\n\...,Fila de exibiçãoFilaFila de exibiçãoFila \nRem...,Fila de exibiçãoFilaFila de exibiçãoFila \nRem...,Fila de exibiçãoFila,,Fila de exibição,Fila de exibiçãoFila \nRemover todosDesconectar,Fila de exibiçãoFila,,,Fila de 

In [16]:
# tentativa de retornar as mesmas colunas consideradas pelo Mario (lembrando que aqui queremos recomendar vídeos sobre finanças)
colunas_selecionadas = ['watch-title', 'watch-view-count', 'watch-time-text','watch7-headline','watch8-sentiment-actions', 
                        'og:image', 'og:image:width','og:description', 'og:video:width', 'og:video:height', 'og:video:tag', 
                        'content_watch-info-tag-list','channel_link_0']

In [17]:
df_raw[colunas_selecionadas].head()

,watch-title,watch-view-count,watch-time-text,watch7-headline,watch8-sentiment-actions,og:image,og:image:width,og:description,og:video:width,og:video:height,og:video:tag,content_watch-info-tag-list,channel_link_0
0,Finanças é Coisa de Criança!,635 visualizações,Publicado em 30 de jul. de 2019,#intusforma #institucional #educacao\n\n\n\n ...,635 visualizações\n\n\n\n\nGostou deste vídeo?...,https://i.ytimg.com/vi/-0HZcKmHXn4/maxresdefau...,1280.0,Um dos nossos primeiros programas que deram vi...,1280.0,720.0,financasecoisadecrianca,Educação,/channel/UCjE76zhvoDw5hnLtmzFHR-Q
1,Câncer 2020 - Profissional e Finanças 1° semestre,37.305 visualizações,Publicado em 3 de dez. de 2019,#tarô #previsão2020 #profissional2020\n\n\n\n ...,37.305 visualizações\n\n\n\n\n\n\n\n2.550\n\nG...,https://i.ytimg.com/vi/-1kS-ZrfcYQ/hqdefault.jpg,480.0,Previsão primeiro semestre de 2020 profissiona...,1280.0,720.0,tarot câncer 2020 profissional,Pessoas e blogs,/channel/UCL6feH5A7tjCoNTKAkJ9ITQ
2,Seja Rica: Conquiste sua Independência Financeira,98 visualizações,Publicado em 30 de mar. de 2017,Seja Rica: Conquiste sua Independência Financeira,98 visualizações\n\n\n\n\n\n\n\n29\n\nGostou d...,https://i.ytimg.com/vi/-40s-kunvQM/hqdefault.jpg,480.0,Seja Rica é o novo quadro aqui do canal. Nesse...,640.0,360.0,Rico,Pessoas e blogs,/channel/UChgFe4WPc8Wx4tJVAdd9-rA
3,Independência FINANCEIRA,41 visualizações,Publicado em 19 de mar. de 2019,Independência FINANCEIRA,41 visualizações\n\n\n\n\n\n\n\n17\n\nGostou d...,https://i.ytimg.com/vi/-6dz-10LkHc/maxresdefau...,1280.0,"Como conseguir sua independência financeira, m...",1280.0,720.0,pouco,Pessoas e blogs,/channel/UCUH-FVOIrY6IW2hs66A4IFA
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Salvando dados
Com o dataframe das features de interesse inicial construído, poderemos partir para as próximas etapas. Para utilizar esses dados em
outro notebook, vamos salvá-lo em formato feather.

In [21]:
df_raw[colunas_selecionadas].to_feather("./datasets/raw_data.feather")

Vale lembrar que nesse ponto, não temos as labels dos vídeos, que serão usadas para criar o modelo. Para proceder com o processo de criação das labels, vamos salvar esses dados no formato csv para utilizar a mesma abordagem do Instrutor.

In [42]:
df_raw[colunas_selecionadas].to_excel("./datasets/raw_data_no_labels.xlsx", engine='xlsxwriter')